# Data Acquisition Notebook
- Notebook detailing ```database.py```
- Database will hold 2 most recent congresses

## ProPublica Data
- Functions and parameters

In [1]:
# API Key
import configparser

# Read/Scrape data
import requests
import json
import re
from bs4 import BeautifulSoup

In [2]:
config = configparser.ConfigParser()
config.read('auth/config.ini')
PROPUBLICA_API_KEY = config.get('propublica', 'PROPUBLICA_API_KEY')

In [3]:
API_ROOT = 'https://api.propublica.org/congress/v1/'
header = {'X-API-Key': f'{PROPUBLICA_API_KEY}'}

In [4]:
def get_house_ids(congress):
    '''
    Function to get house members' ProPublica ID by congress number
    '''

    call_string = API_ROOT + f'{congress}/house/members.json'
    r = requests.get(call_string, headers=header)
    result = r.json()['results'][0]['members']
    member_ids = [ member['id'] for member in result ]
    
    return member_ids

In [5]:
def get_member_data(member):
    '''
    Function to get house member's data
    '''
    
    call_string = API_ROOT + f'members/{member}.json'
    r = requests.get(call_string, headers=header)
    result = r.json()['results'][0]
    
    return result

## Get Initial Congress and Clean Data/Select Features

In [16]:
# House members of the 116th Congress (2018-2020)
ids_116 = get_house_ids(116)
members_116 = [ get_member_data(member) for member in ids_116 ]

In [17]:
from pprint import pprint

In [18]:
def member_cleaner(member):
    '''
    Function to keep relevent information on congress member
    '''
    
    roles = member['roles']
    roles.sort(key=lambda x: x['congress'], reverse=True)
    fec_id = roles[0]['fec_candidate_id']
    
    mem_dict = {
        '_id': member['id'],
        'bio': {
            'first_name': member['first_name'],
            'middle_name': member['middle_name'],
            'last_name': member['last_name'],
            'dob': member['date_of_birth'],
            'gender': member['gender'],
            'current_party': member['current_party'],
        },
        'activity': {
            'most_recent_vote': member['most_recent_vote'],
            'last_updated': member['last_updated'],
            'in_office': member['in_office'],
        },
        'other_ids': {
            'google_id': member['google_entity_id'],
            'votesmart_id': member['votesmart_id'],
            'govtrack_id': member['govtrack_id'],
            'cspan_id': member['cspan_id'],
            'crp_id': member['crp_id'],
            'fec_id': fec_id,
        },
        'roles': member['roles'],   
    }
    
    return mem_dict

In [19]:
members_insert = map(member_cleaner, members_116)

In [10]:
# Database connection and authentication
import firebase_admin
from firebase_admin import credentials, firestore
from google.cloud.firestore_v1.batch import WriteBatch

In [11]:
GCP_AUTH_PATH = config.get('firebase', 'GCP_AUTH_PATH')
cred = credentials.Certificate(GCP_AUTH_PATH)
app = firebase_admin.initialize_app(cred)

In [20]:
# Database
db = firestore.client()

In [21]:
def batch_insert_members(members, db):
    '''
    Function to batch insert house members into database
    '''
    
    batch = db.batch()
    for member in members:
        try:
            _id = member['_id']
            insert_ref = db.collection("reps").document(f"{_id}")
            batch.set(insert_ref, member)
        except:
            print(member['_id'])
            
    batch.commit()
    
    return len(batch.write_results)

In [22]:
batch_insert_members(members_insert, db)

451

In [23]:
len(members_116)

451

In [ ]:
rep_ref = db.collection('reps').document('A000370')
result = rep_ref.get().to_dict()

## Get Number of Current Congress

In [ ]:
from datetime import datetime

In [ ]:
# Year of 1st Congress
begin = 1789

# Determine year to use
now = datetime.now().date()
if now.day >= 20:
    year = now.year
else:
    year = now.year - 1
    
# Determine current congress
current_cong_num = int(1 + (year - begin)/2)

In [ ]:
print('Current Congress:', current_cong_num)

In [ ]:
# House members of current congress
current_members = get_house_ids(current_cong_num)
members = [ get_member_data(mem) for mem in current_members ]

In [ ]:
member = members[0]
sample_dict(member)

In [ ]:
def get_roll_call_vote(congress, session_number, roll_call_num):
    query = f'{congress}/house/sessions/{session_number}/votes/{roll_call_num}.json'
    call_string = API_ROOT + query
    r = requests.get(call_string, headers=header)
    result = r.json()['results']['votes']
    
    return result